In [1]:
import json
import cv2
import pandas as pd
import os
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [2]:
results_file = './results.json'
labels_dir = './test/unit_labels/test/'
img_dir = './test/images/test/'
models_dir = './models/'

In [3]:
with open(results_file, 'r') as fp:
    results = json.load(fp)

In [5]:
# map_models = {}
# for file in os.scandir(models_dir):
#     if file.is_file() and file.name.startswith('model') and file.name.endswith('.pt'):
#         model_name = file.name.split('model_')[1][:-3]
#         if 'yolo' not in model_name:
#             map_models[model_name] = len(map_models)

units = [['infantry',
          'anti_tank',
          'armour',
          'wheeled',
          'unit_tactical'],
         ['recce',
          'medic',
          'signal',
          'hq_unit',
          'supply',
          'artillery',
          'mortar',
          'air_defence'],
         ['infantry',
          'anti_tank',
          'recce',
          'medic',
          'signal'],
         ['motorized', 'cannon'],
         ['team', 'squad', 'half-platoon', 'platoon', 'company',  # Currently repetition because unit sizes are sampled with uniform distirubution
          'battalion']]  # 'brigade', 'regiment', 'division']

units = set([j for sub in units for j in sub])

map_models = {}
for unit in units:
    map_models[unit] = len(map_models)

In [6]:
for i, row in enumerate(results):
    for s in row['symbols']:
        s['used'] = 0

column_names = ["actual", "predicted", "correct",
                "incorrect", "missed"]  # , "x1", "x2", "y1", "y2"]
df = pd.DataFrame(columns=column_names)
for i, row in enumerate(results):
    cur_img = row['img'].strip()
    img = cv2.imread(f"{img_dir}{cur_img}")
    img_txt = cur_img.split('.')[0] + '.txt'
    with open(f'{labels_dir}{img_txt}') as fp:
        for line in fp:
            splits = line.strip().split(" ")
            x_c, y_c, h, w = map(float, splits[1:])
            actual = splits[0].strip().split("__")[1:]
            height, width, channels = img.shape
            x_c, y_c, w, h = float(x_c)*width, float(y_c) * \
                height, float(w)*width, float(h)*height

            predicted = []
            for s in row['symbols']:
                if s['used'] == 0:
                    if s['xmin'] <= x_c and x_c <= s['xmax'] and s['ymin'] <= y_c and y_c <= s['ymax']:
                        predicted = s['labels']
                        s['used'] = 1
                        break

            dif = set(actual) - set(predicted)
            missed = len(dif)
            correct = len(actual) - missed
            incorrect = len(predicted) - correct
            # print(map_labels(map_models, actual))
            # print(map_labels(map_models, predicted))

            df = pd.concat([df, pd.DataFrame(
                [[actual, predicted, correct, incorrect, missed]], columns=column_names)], ignore_index=True)

for row in results:
    for s in row['symbols']:
        if s['used'] == 0:
            df = pd.concat([df, pd.DataFrame([[[], s['labels'], 0, 0, len(
                s['labels'])]], columns=column_names)], ignore_index=True)

In [7]:
df.head()

,actual,predicted,correct,incorrect,missed
0,"[mortar, company]","[mortar, company]",2,0,0
1,"[supply, armour, company]","[armour, company]",2,0,1
2,"[armour, half-platoon]",[company],0,1,2
3,"[infantry, company]",[company],1,0,1
4,"[supply, squad]",[company],0,1,2


In [8]:
# actual_labels = set([j for sub in df[['actual', 'predicted']].apply(lambda x: x[0] + x[1], axis=1).to_list() for j in sub])
# for label in actual_labels:
#     if label not in map_models:
#         map_models[label] = len(map_models)

In [9]:
def map_labels(map_models, input_list):
    out = [0] * len(map_models)
    for label in input_list:
        if label in map_models:
            out[map_models[label]] = 1
        elif label == 'anti-tank':
            out[map_models['anti_tank']] = 1
        else:
            # out[map_models['other']] = 1
            raise ValueError(f'Add label {label}')
    return out


df['actual_encoded'] = df['actual'].apply(lambda x: map_labels(map_models, x))
df['predicted_encoded'] = df['predicted'].apply(lambda x: map_labels(map_models, x))

In [10]:
df.head()

,actual,predicted,correct,incorrect,missed,actual_encoded,predicted_encoded
0,"[mortar, company]","[mortar, company]",2,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[supply, armour, company]","[armour, company]",2,0,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,"[armour, half-platoon]",[company],0,1,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[infantry, company]",[company],1,0,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[supply, squad]",[company],0,1,2,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [11]:
# y_true = np.sum(df['actual_encoded'].to_list(), axis=0)
# y_pred = np.sum(df['predicted_encoded'].to_list(), axis=0)
y_true = df['actual_encoded'].tolist()
y_pred = df['predicted_encoded'].tolist()

In [12]:
sorted_dict = sorted(map_models.items(), key=lambda x: x[1])
sorted_labels = [item[0] for item in sorted_dict]

In [13]:
y_true = np.array(y_true)
# y_true

In [14]:
y_pred = np.array(y_pred)
# y_pred

In [15]:
conf_mat_dict={}
report_dict = {}

for label_col in range(len(sorted_labels)):
    y_true_label = y_true[:, label_col]
    y_pred_label = y_pred[:, label_col]
    conf_mat_dict[sorted_labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    report_dict[sorted_labels[label_col]] = classification_report(y_pred=y_pred_label, y_true=y_true_label)


In [21]:
for label, matrix in conf_mat_dict.items():
    print("Confusion matrix for label {}:".format(label))
    print(matrix)

Confusion matrix for label infantry:
[[238   0]
 [ 42   0]]
Confusion matrix for label supply:
[[252   0]
 [ 28   0]]
Confusion matrix for label platoon:
[[212   0]
 [ 64   4]]
Confusion matrix for label artillery:
[[269   0]
 [  2   9]]
Confusion matrix for label medic:
[[260   0]
 [ 20   0]]
Confusion matrix for label air_defence:
[[267   0]
 [ 13   0]]
Confusion matrix for label unit_tactical:
[[271   0]
 [  9   0]]
Confusion matrix for label mortar:
[[262   4]
 [  5   9]]
Confusion matrix for label cannon:
[[245   0]
 [ 35   0]]
Confusion matrix for label anti_tank:
[[242   0]
 [ 12  26]]
Confusion matrix for label team:
[[241   0]
 [ 38   1]]
Confusion matrix for label recce:
[[260   0]
 [ 20   0]]
Confusion matrix for label squad:
[[240   0]
 [ 39   1]]
Confusion matrix for label battalion:
[[241  21]
 [ 15   3]]
Confusion matrix for label armour:
[[195   0]
 [ 34  51]]
Confusion matrix for label signal:
[[262   0]
 [ 18   0]]
Confusion matrix for label motorized:
[[254   0]
 [ 2

In [23]:
for label, matrix in report_dict.items():
    print("Classification report matrix for label {}:".format(label))
    print(matrix)

Classification report matrix for label infantry:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       238
           1       0.00      0.00      0.00        42

    accuracy                           0.85       280
   macro avg       0.42      0.50      0.46       280
weighted avg       0.72      0.85      0.78       280

Classification report matrix for label supply:
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       252
           1       0.00      0.00      0.00        28

    accuracy                           0.90       280
   macro avg       0.45      0.50      0.47       280
weighted avg       0.81      0.90      0.85       280

Classification report matrix for label platoon:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87       212
           1       1.00      0.06      0.11        68

    accuracy                          